In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import xarray as xr
import numpy as np
from natsort import natsorted
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [14]:
import sys
original_path = os.getcwd()  
os.chdir(original_path)
function_path = './functions/'
sys.path.append(function_path)
from smoothing_function import *
from model_analysis_function import *
from analysis_function import *
from Plot_function import *
from ar6_area_weighted_cont_global import *
from f1_0_H_sum_multi_region_constrain import *

# from obs_function import *

### Constrain the regional, continent and global land warming

### Load the regional, continent and global land data

In [15]:
path = './saved_data/'

def load_xr_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

obs_all = load_xr_pickle(path + '0.1.HadCRUT5.Tas.anomalies.46AR6regions_7cont_1glob_175years_1850-2024.pkl')

obs_200runs_all = load_xr_pickle(path + '0.2.HadCRUT5.200runs.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_175years_1850-2024.pkl')

mod_all_his_fu = load_xr_pickle(path + '0.4.Smoothed_His-Fu-ALL.25mods.mean.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_251years_1850-2100.pkl')

ln_mod_all = load_xr_pickle(path + '0.3.Large_ensembles.320runs.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_251years_1850-2100.pkl')

mod_45_pseudo = load_xr_pickle(path + '0.5.pseudo-model.15mod.run1-3.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_176years_1850-2025.pkl')

print_xarray_info(obs_all, obs_200runs_all, mod_all_his_fu, ln_mod_all, mod_45_pseudo)


Array 1:
  Sizes: Frozen({'year': 175, 'region': 54})
  Coords: ['realization', 'year', 'region', 'abbrevs', 'names']

Array 2:
  Sizes: Frozen({'runs': 200, 'year': 175, 'region': 54})
  Coords: ['runs', 'year', 'region', 'abbrevs', 'names']

Array 3:
  Sizes: Frozen({'forcing': 1, 'model_name': 25, 'year': 251, 'region': 54})
  Coords: ['model_name', 'region', 'year', 'abbrevs', 'names', 'forcing']

Array 4:
  Sizes: Frozen({'model_run': 320, 'year': 251, 'region': 54})
  Coords: ['year', 'model_run', 'model_name', 'region', 'abbrevs', 'names']

Array 5:
  Sizes: Frozen({'model_run': 45, 'year': 176, 'region': 54})
  Coords: ['model_run', 'year', 'model_name', 'region', 'abbrevs', 'names']


In [16]:
from f_schemes import *

region_groups = {
    'North and Central America': ['GIC', 'NWN', 'NEN', 'WNA', 'CNA', 'ENA', 'NCA', 'SCA', 'CAR'],
    'South America': ['NWS', 'NSA', 'NES', 'SAM', 'SWS', 'SES', 'SSA'],
    'Europe': ['NEU', 'WCE', 'EEU', 'MED'],
    'Africa': ['SAH', 'WAF', 'CAF', 'NEAF', 'SEAF', 'WSAF', 'ESAF', 'MDG'],
    'Asia': ['RAR', 'WSB', 'ESB', 'RFE', 'WCA', 'ECA', 'TIB', 'EAS', 'ARP', 'SAS'],
    'Australasia': ['SEA', 'NAU', 'CAU', 'EAU', 'SAU', 'NZ'],
    'Antarctica': ['EAN', 'WAN'],
    'Global land': ['LSAT']
}

scheme_reg_lsat = generate_global_regional_target_constraint_pairs(region_groups, global_abbrev='LSAT')

scheme_reg_lsat

[{'target_reg': 'GIC', 'constrain_used_reg': ['GIC', 'LSAT']},
 {'target_reg': 'NWN', 'constrain_used_reg': ['NWN', 'LSAT']},
 {'target_reg': 'NEN', 'constrain_used_reg': ['NEN', 'LSAT']},
 {'target_reg': 'WNA', 'constrain_used_reg': ['WNA', 'LSAT']},
 {'target_reg': 'CNA', 'constrain_used_reg': ['CNA', 'LSAT']},
 {'target_reg': 'ENA', 'constrain_used_reg': ['ENA', 'LSAT']},
 {'target_reg': 'NCA', 'constrain_used_reg': ['NCA', 'LSAT']},
 {'target_reg': 'SCA', 'constrain_used_reg': ['SCA', 'LSAT']},
 {'target_reg': 'CAR', 'constrain_used_reg': ['CAR', 'LSAT']},
 {'target_reg': 'NWS', 'constrain_used_reg': ['NWS', 'LSAT']},
 {'target_reg': 'NSA', 'constrain_used_reg': ['NSA', 'LSAT']},
 {'target_reg': 'NES', 'constrain_used_reg': ['NES', 'LSAT']},
 {'target_reg': 'SAM', 'constrain_used_reg': ['SAM', 'LSAT']},
 {'target_reg': 'SWS', 'constrain_used_reg': ['SWS', 'LSAT']},
 {'target_reg': 'SES', 'constrain_used_reg': ['SES', 'LSAT']},
 {'target_reg': 'SSA', 'constrain_used_reg': ['SSA', 'L

### Constrained warming relative to current year 2025

In [17]:
#### global_region constraint
mod_all_his_fu_new = xr.concat([mod_all_his_fu.isel(region=slice(0, 46)), mod_all_his_fu.isel(region=53)], dim = 'region')

constrain_func = constrain_sum_reg
obs = obs_all
obs_200runs = obs_200runs_all
ln_mod = ln_mod_all
mod_his = mod_all_his_fu_new
mod_da = mod_all_his_fu_new
obs_ar6 = obs_all
region_names = list(mod_da.names.values)
forcing_list = mod_da.forcing  # xarray.DataArray
his_forcing = ['ALL']
constrain_forcing_names=['ALL']

scheme_pairs =  scheme_reg_lsat

_, _, prior_smooth_ALL_ref2025, post_smooth_ALL_ref2025, obs_adj_ALL_ref2025 = \
    process_all_regions(scheme_pairs, constrain_func, obs, obs_200runs, ln_mod, mod_his, mod_da, obs_ar6, region_names, forcing_list, his_forcing, constrain_forcing_names, 
    reg_id = slice(0, len(region_names)),
    uncertainty_ref_period=(1850, 2025),
    ref_period=(2025, 2026),
    obs_adjust_ref_period=(1961, 2025),
    warming_target_period=(2016, 2025),
    calc_smoothed = True, 
    print_constraint_regions = True)




Processing region 0: Greenland/Iceland
 Constraining used region 0: ['GIC', 'LSAT']
Processing region 1: N.W.North-America
 Constraining used region 1: ['NWN', 'LSAT']
Processing region 2: N.E.North-America
 Constraining used region 2: ['NEN', 'LSAT']
Processing region 3: W.North-America
 Constraining used region 3: ['WNA', 'LSAT']
Processing region 4: C.North-America
 Constraining used region 4: ['CNA', 'LSAT']
Processing region 5: E.North-America
 Constraining used region 5: ['ENA', 'LSAT']
Processing region 6: N.Central-America
 Constraining used region 6: ['NCA', 'LSAT']
Processing region 7: S.Central-America
 Constraining used region 7: ['SCA', 'LSAT']
Processing region 8: Caribbean
 Constraining used region 8: ['CAR', 'LSAT']
Processing region 9: N.W.South-America
 Constraining used region 9: ['NWS', 'LSAT']
Processing region 10: N.South-America
 Constraining used region 10: ['NSA', 'LSAT']
Processing region 11: N.E.South-America
 Constraining used region 11: ['NES', 'LSAT']
Proc

### Constrained warming relative to pre-industrial 1850-1900 year

In [18]:
#### global_region constraint
constrain_func = constrain_sum_reg
obs = obs_all
obs_200runs = obs_200runs_all
ln_mod = ln_mod_all
mod_his = mod_all_his_fu_new
mod_da = mod_all_his_fu_new
obs_ar6 = obs_all
region_names = list(mod_da.names.values)
forcing_list = mod_da.forcing  # xarray.DataArray
his_forcing = ['ALL']
constrain_forcing_names=['ALL']

scheme_pairs =  scheme_reg_lsat

_, _, prior_smooth_ALL_ref1850, post_smooth_ALL_ref1850, obs_adj_ALL_ref1850 = \
    process_all_regions(scheme_pairs, constrain_func, obs, obs_200runs, ln_mod, mod_his, mod_da, obs_ar6, region_names, forcing_list, his_forcing, constrain_forcing_names, 
    reg_id = slice(0, len(region_names)),
    uncertainty_ref_period=(1850, 2025),
    ref_period=(1850, 1900),
    obs_adjust_ref_period=(1961, 2025),
    warming_target_period=(2016, 2025),
    calc_smoothed = True, 
    print_constraint_regions = True)




Processing region 0: Greenland/Iceland
 Constraining used region 0: ['GIC', 'LSAT']
Processing region 1: N.W.North-America
 Constraining used region 1: ['NWN', 'LSAT']
Processing region 2: N.E.North-America
 Constraining used region 2: ['NEN', 'LSAT']
Processing region 3: W.North-America
 Constraining used region 3: ['WNA', 'LSAT']
Processing region 4: C.North-America
 Constraining used region 4: ['CNA', 'LSAT']
Processing region 5: E.North-America
 Constraining used region 5: ['ENA', 'LSAT']
Processing region 6: N.Central-America
 Constraining used region 6: ['NCA', 'LSAT']
Processing region 7: S.Central-America
 Constraining used region 7: ['SCA', 'LSAT']
Processing region 8: Caribbean
 Constraining used region 8: ['CAR', 'LSAT']
Processing region 9: N.W.South-America
 Constraining used region 9: ['NWS', 'LSAT']
Processing region 10: N.South-America
 Constraining used region 10: ['NSA', 'LSAT']
Processing region 11: N.E.South-America
 Constraining used region 11: ['NES', 'LSAT']
Proc

In [19]:
prior_smooth_ALL_ref1850

<xarray.DataArray 'prior_mean_5_95_smooth' (region: 47, forcing: 1,
                                            quantile: 3, year: 251)> Size: 283kB
array([[[[ 4.35199256e-02,  4.11738343e-02,  3.88278147e-02, ...,
           7.73796627e+00,  7.80760454e+00,  7.87724281e+00],
         [-6.32209229e-01, -6.26804201e-01, -6.21400478e-01, ...,
           3.69275845e+00,  3.72640078e+00,  3.76004312e+00],
         [ 7.24482153e-01,  7.13780598e-01,  7.03080751e-01, ...,
           1.17177418e+01,  1.18215515e+01,  1.19253612e+01]]],


       [[[ 2.33000252e-02,  2.45371039e-02,  2.57648778e-02, ...,
           9.11123695e+00,  9.19707565e+00,  9.28291436e+00],
         [-4.41929870e-01, -4.36676729e-01, -4.31428331e-01, ...,
           5.23768849e+00,  5.28530041e+00,  5.33291233e+00],
         [ 4.83294886e-01,  4.80247852e-01,  4.77200819e-01, ...,
           1.29638270e+01,  1.30872970e+01,  1.32107670e+01]]],


       [[[ 2.53000364e-03,  6.06587705e-03,  9.59424219e-03, ...,
           9.87800668e+00,  9.97012858e+00,  1.00622505e+01],
         [-6.48318200e-01, -6.35978987e-01, -6.23639774e-01, ...,
           5.64850801e+00,  5.69878658e+00,  5.74906514e+00],
...
           2.79598255e+00,  2.82054741e+00,  2.84511228e+00],
         [ 2.17638761e-01,  2.22353063e-01,  2.27064476e-01, ...,
           7.53615031e+00,  7.60741816e+00,  7.67868600e+00]]],


       [[[-8.23897804e-02, -7.77259768e-02, -7.30642063e-02, ...,
           5.32937730e+00,  5.37801604e+00,  5.42665477e+00],
         [-6.47422516e-01, -6.34323742e-01, -6.21229899e-01, ...,
           2.04534275e+00,  2.06127788e+00,  2.07721301e+00],
         [ 5.07244668e-01,  5.00366127e-01,  4.93494771e-01, ...,
           8.65346566e+00,  8.73592465e+00,  8.81838364e+00]]],


       [[[ 3.74299521e-02,  3.63245991e-02,  3.52179423e-02, ...,
           5.88389667e+00,  5.93846288e+00,  5.99302908e+00],
         [-2.13136197e-01, -2.10669790e-01, -2.08203814e-01, ...,
           3.72550077e+00,  3.75867109e+00,  3.79184142e+00],
         [ 2.83234662e-01,  2.79238872e-01,  2.75243031e-01, ...,
           8.14935224e+00,  8.22847029e+00,  8.30758834e+00]]]],
      shape=(47, 1, 3, 251))
Coordinates:
  * region       (region) int64 376B 0 1 2 3 4 5 6 7 ... 39 40 41 42 43 44 45 46
  * year         (year) int64 2kB 1850 1851 1852 1853 ... 2097 2098 2099 2100
  * forcing      (forcing) <U3 12B 'ALL'
  * quantile     (quantile) <U4 48B 'mean' '5th' '95th'
    abbrevs      (region) <U4 752B 'GIC' 'NWN' 'NEN' ... 'EAN' 'WAN' 'LSAT'
    names        (region) <U25 5kB 'Greenland/Iceland' ... 'Global Land'
    realization  int64 8B 100

### Save results to files

In [20]:
save_path = './saved_data/'
name = '2.2_constrained_46reg_lsat_smoothed_warming_ref2025_1850-2100.pkl'

with open(save_path + name, 'wb') as wi:
	pickle.dump(post_smooth_ALL_ref2025, wi)

name = '2.3_constrained_46reg_lsat_smoothed_warming_ref1850_1850-2100.pkl'

with open(save_path + name, 'wb') as wi:
	pickle.dump(post_smooth_ALL_ref1850, wi)